# deeplodocus-learn

#### За основу было взято исследование [Learning to Play Chess with Minimal Lookahead and Deep Value Neural Networks](https://www.ai.rug.nl/~mwiering/Thesis_Matthia_Sabatelli.pdf)

#### Мои итоговые датасеты и обученные модели: [Google drive](https://drive.google.com/drive/folders/1VvwrBkgA9i_8i3D2PHZ_CJ_ni2xFUPlT)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, BatchNormalization
import keras
import pandas as pd
import stockfish
import chess, chess.pgn
import io
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np

# https://github.com/paintception/DeepChess/
X = np.load('final/Positions.npy')
y = np.load('final/Labels.npy')

In [ ]:
model = Sequential()
 
model.add(Dense(2048, activation='elu', input_shape=(768,)))
model.add(BatchNormalization())

model.add(Dense(2048, activation='elu'))
model.add(BatchNormalization())

model.add(Dense(2048, activation='elu'))
model.add(BatchNormalization())

model.add(Dense(1, activation='linear'))

model.summary()

sgd = keras.optimizers.legacy.SGD(learning_rate=0.001, decay=1e-8, momentum=0.7, nesterov=True)
model.compile(loss='mean_squared_error', optimizer=sgd)

In [ ]:
history = model.fit(X, y, batch_size=248, epochs=500)

model.save('1_400_000_ep500.h5')